In [37]:
import pandas as pd
import useful_functions as use
from fuzzywuzzy import fuzz

C:\Users\vigrose\AppData\Local\Continuum\anaconda3\lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [2]:
humach = pd.read_csv('../../Data/PhoneAppend/humach_results.csv')
thing_1 = pd.read_excel('../../Data/PhoneAppend/Bing_BOLO_Humach_Sample_25-05-21.xlsx')
thing_2 = pd.read_excel('../../Data/DataGov/National_Downloadable_File_BOLO_Humach_Sample_25-05-21.xlsx')
thing_3 = pd.read_excel('../../Data/DataGov/National_Downloadable_File_Humach_Random_Sample_25-05-21.xlsx')

In [3]:
thing_3 = thing_3.rename(columns = {'POLO_MAILING_ADDRESS_1':'POLO_MAILING_LINE_1', 'POLO_MAILING_ADDRESS_2':'POLO_MAILING_LINE_2'})

In [4]:
og = pd.concat([thing_1, thing_2, thing_3])

C:\Users\vigrose\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


In [5]:
humach['ME']=use.fix_me(humach.PHYSICIAN_ME_NUMBER)
og['ME']=use.fix_me(og.ME)

In [6]:
ALL_THINGS = pd.merge(og, humach, on='ME', suffixes = (['_UPDATED','_OG']))

In [7]:
updated = ALL_THINGS[(ALL_THINGS.COMMENTS=='COMPLETE')&(ALL_THINGS.OFFICE_ADDRESS_VERIFIED_UPDATED==2)]

In [8]:
def is_a_match(thing_1, thing_2):
    thing_1 = thing_1.upper().strip().replace('  ',' ')
    thing_2 = thing_2.upper().strip().replace('  ',' ')
    if thing_1 == thing_2:
        match = True
#     elif thing_1 in thing_2:
#         match = True
#     elif thing_2 in thing_1:
#         match = True
#     elif fuzz.ratio(thing_1, thing_2)>threshold:
#         match = True
#         print('')
#         print(fuzz.ratio(thing_1, thing_2))
#         print('------------------------------------')
    else:
        match = False
    return(match)

In [36]:
def is_a_fuzzy_match(thing_1, thing_2, threshold):
    thing_1 = thing_1.upper().strip().replace('  ',' ')
    thing_2 = thing_2.upper().strip().replace('  ',' ')
    if thing_1 == thing_2:
        match = True
    elif thing_1 in thing_2:
        match = True
    elif thing_2 in thing_1:
        match = True
    elif fuzz.ratio(thing_1, thing_2)>threshold:
        match = True
        print('')
        print(fuzz.ratio(thing_1, thing_2))
        print('------------------------------------')
    else:
        match = False
    return(match)

In [9]:
updated = updated.fillna('None')

In [10]:
updated['POLO_SHORT_ZIP'] = [use.fix_zipcode(x) for x in updated.POLO_ZIP]

In [11]:
updated['OFFICE_SHORT_ZIP'] = [use.fix_zipcode(x) for x in updated.OFFICE_ADDRESS_ZIP]

In [20]:
pure_matches = []
city_changes = []
truncated_zips = []
address_2_changes =  []
address_swaps = []

for row in updated.itertuples():
    pure_match = False
    city_change = False
    truncated_zip = False
    address_2_change =  False
    address_swap = False
    if is_a_match(row.POLO_MAILING_LINE_2, row.OFFICE_ADDRESS_LINE_2) and is_a_match(row.POLO_MAILING_LINE_1, row.OFFICE_ADDRESS_LINE_1):
            if row.POLO_ZIP == row.OFFICE_ADDRESS_ZIP and is_a_match(row.POLO_CITY, row.OFFICE_ADDRESS_CITY):
                pure_match = True
            elif row.POLO_ZIP == row.OFFICE_ADDRESS_ZIP:
                city_change = True
            elif row.POLO_SHORT_ZIP == row.OFFICE_SHORT_ZIP and is_a_match(row.POLO_CITY, row.OFFICE_ADDRESS_CITY):
                truncated_zip = True
    elif is_a_match(row.POLO_MAILING_LINE_2, row.OFFICE_ADDRESS_LINE_2) and row.POLO_ZIP == row.OFFICE_ADDRESS_ZIP and is_a_match(row.POLO_CITY, row.OFFICE_ADDRESS_CITY):
        address_2_change = True
    elif is_a_match(row.POLO_MAILING_LINE_2, row.OFFICE_ADDRESS_LINE_2) and is_a_match(row.POLO_MAILING_LINE_1, row.OFFICE_ADDRESS_LINE_2):
        address_swap = True
    pure_matches.append(pure_match)
    city_changes.append(city_change)
    truncated_zips.append(truncated_zip)
    address_2_changes.append(address_2_change)
    address_swaps.append(address_swap)

77305 77304
00605 00603
86324 86326


In [38]:
pure_matches = []
city_changes = []
truncated_zips = []
address_2_changes =  []
address_swaps = []
fuzzy_matches = []

for row in updated.itertuples():
    if is_a_match(row.POLO_MAILING_LINE_2, row.OFFICE_ADDRESS_LINE_2) and is_a_match(row.POLO_MAILING_LINE_1, row.OFFICE_ADDRESS_LINE_1):
            if row.POLO_ZIP == row.OFFICE_ADDRESS_ZIP and is_a_match(row.POLO_CITY, row.OFFICE_ADDRESS_CITY):
                pure_matches.append(row.ME)
            elif row.POLO_ZIP == row.OFFICE_ADDRESS_ZIP:
                city_changes.append(row.ME)
            elif row.POLO_SHORT_ZIP == row.OFFICE_SHORT_ZIP and is_a_match(row.POLO_CITY, row.OFFICE_ADDRESS_CITY):
                truncated_zips.append(row.ME)
    elif is_a_match(row.POLO_MAILING_LINE_2, row.OFFICE_ADDRESS_LINE_2) and row.POLO_ZIP == row.OFFICE_ADDRESS_ZIP and is_a_match(row.POLO_CITY, row.OFFICE_ADDRESS_CITY):
        address_2_changes.append(row.ME)
    elif is_a_match(row.POLO_MAILING_LINE_2, row.OFFICE_ADDRESS_LINE_2) and is_a_match(row.POLO_MAILING_LINE_1, row.OFFICE_ADDRESS_LINE_2):
        address_swaps.append(row.ME)
    elif is_a_fuzzy_match(row.POLO_MAILING_LINE_2, row.OFFICE_ADDRESS_LINE_2, 85) and row.POLO_SHORT_ZIP == row.OFFICE_SHORT_ZIP:
        fuzzy_matches.append(row.ME)


98
------------------------------------

97
------------------------------------

95
------------------------------------

98
------------------------------------

92
------------------------------------

93
------------------------------------

93
------------------------------------

86
------------------------------------

94
------------------------------------

95
------------------------------------

86
------------------------------------

91
------------------------------------

96
------------------------------------

90
------------------------------------

90
------------------------------------

97
------------------------------------

91
------------------------------------

87
------------------------------------

90
------------------------------------

92
------------------------------------

94
------------------------------------

96
------------------------------------

88
------------------------------------

91
------------------------------------

97
------------

In [40]:
questionable_updates = pure_matches + city_changes+truncated_zips+address_2_changes+address_swaps+fuzzy_matches

In [46]:
list_match = []
for me in updated.ME:
    if me in pure_matches:
        match = 'EXACT MATCH'
    elif me in city_changes:
        match = 'CITY SPELLING'
    elif me in truncated_zips:
        match = 'SHORTENED ZIPCODE'
    elif me in address_2_changes:
        match = 'DIFFERENT ADDRESS_1'
    elif me in fuzzy_matches:
        match = 'FUZZY MATCH'
    else:
        match = 'None'
    list_match.append(match)

In [48]:
updated['NOTES'] = list_match

In [49]:
updated

,DESCRIPTION,FAX_NUMBER,FIRST_NAME,LAST_NAME,ME,MIDDLE_NAME,PE_CD,POLO_CITY,POLO_MAILING_LINE_1,POLO_MAILING_LINE_2,...,WSLIVE_FILE_DT,MATCH_ADDR,MATCH_PHONE,MATCH_ADDR_LONG,SPECIALTY,SPECIALTY_UPDATED,SOURCE_FILE_DT,POLO_SHORT_ZIP,OFFICE_SHORT_ZIP,NOTES
8,Group Practice,8.03774e+09,IKE,STEWART,04114020970,CHRISTOPHER,30,Sumter,None,220 Broad St,...,06/11/2021,041140209730429150,04114020978037735227,None,FM,1,05/26/2021,29150,29150,None
10,Self-Employed Solo Practice ...,7.6078e+09,DAVID,WALDMAN,03509721393,JOHN,11,Rancho Mirage,None,39000 Bob Hope Dr # 100,...,06/11/2021,03509721393900092270,03509721397605683595,None,A,1,05/26/2021,92270,92270,FUZZY MATCH
15,Self-Employed Solo Practice ...,9.41954e+09,CLAYTON,BREDLAU,03841780182,EARL,11,Sarasota,None,1851 Hawthorne St,...,06/11/2021,0384178018121734239,03841780189413650433,None,CD,1,05/26/2021,34239,34239,None
18,Group Practice,2.03794e+09,KAREN,COBLENS,03545870184,ELISE,30,Danbury,None,132 Main St,...,06/18/2021,035458701810006810,03545870182037941979,None,IM,1,05/26/2021,06810,06810,None
24,No Classification,5.62283e+09,ANDREW,MESSIHA,04114090871,HANY,110,Laguna Hills,None,23521 Paseo De Valencia Ste 204,...,06/04/2021,04114090872352192653,04114090879494582026,None,PMM,1,05/26/2021,92653,92653,FUZZY MATCH
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3556,Two Physician Practice-Full Or Part Owner,9.78534e+09,DANIEL,O'LEARY,02416850774,JOHN,13,LEOMINSTER,SUITE 205,50 MEMORIAL DR,...,06/11/2021,02416850775001453,02416850779785344241,None,IM,1,05/26/2021,01453,01453,FUZZY MATCH
3557,No Classification,None,DANIEL,CIESLAK,05606140365,ALBERT,110,SANDY,None,955 E 11400,...,06/11/2021,056061403695584094,05606140368015710030,None,US,1,05/26/2021,84094,84094,FUZZY MATCH
3564,Group Practice,8.59277e+09,KENNETH,HUGHES,03901890616,V,30,LEXINGTON,SUITE 230,230 FOUNTAIN CT,...,06/18/2021,0390189061172040503,03901890618592781114,None,OTO,1,05/26/2021,40509,40503,None
3566,No Classification,None,ELIZABETH,HUMSTON,05175080670,GORDON,110,JOHNSON CITY,None,303 MED TECH PKWY,...,06/11/2021,051750806730337604,05175080674237945520,None,FM,1,05/26/2021,37604,37604,FUZZY MATCH


In [51]:
updated[['ME','NOTES','POLO_CITY', 'POLO_MAILING_LINE_1',
       'POLO_MAILING_LINE_2', 'POLO_STATE', 'POLO_ZIP', 'OFFICE_ADDRESS_LINE_1', 'OFFICE_ADDRESS_LINE_2',
       'OFFICE_ADDRESS_CITY', 'OFFICE_ADDRESS_STATE', 'OFFICE_ADDRESS_ZIP','Source']].to_csv('../../Data/Humach/Questionable_Address_Updates.csv', index=False)

In [23]:
len(updated[address_2_changes][['POLO_CITY', 'POLO_MAILING_LINE_1',
       'POLO_MAILING_LINE_2', 'POLO_STATE', 'POLO_ZIP', 'OFFICE_ADDRESS_LINE_1', 'OFFICE_ADDRESS_LINE_2',
       'OFFICE_ADDRESS_CITY', 'OFFICE_ADDRESS_STATE', 'OFFICE_ADDRESS_ZIP','Source']])

28

In [ ]:
updated[updated.OFFICE_ADDRESS_LINE_2=='2675 N DECATUR RD'][['POLO_CITY', 'POLO_MAILING_LINE_1',
       'POLO_MAILING_LINE_2', 'POLO_STATE', 'POLO_ZIP', 'OFFICE_ADDRESS_LINE_1', 'OFFICE_ADDRESS_LINE_2',
       'OFFICE_ADDRESS_CITY', 'OFFICE_ADDRESS_STATE', 'OFFICE_ADDRESS_ZIP','Source']]

In [24]:
len(updated)

731

In [43]:
269/731*100

36.79890560875513

In [42]:
len(fuzzy_matches)

221

## Exact Match
### 11 (1.50%)
# Truncated Zipcode
### 196 (26.81%)
# Spelled Out City Name
### 6 (0.82%)
# Different Suite
### 28 (8.83%)
## Fuzzy Matches
### 269 (36.79%)